# 멀티 호라이즌 섹터 예측 + 호라이즌별 인더스트리 클러스터링

## 워크플로우
1. 멀티 호라이즌 예측 (1d, 3d, 1w, 1m, 1q, 6m, 1y)
2. **각 호라이즌별로 독립적으로:**
   - Top-3 섹터 선정
   - 해당 섹터 내 인더스트리 클러스터링 (KMeans, 4개 클러스터)
   - 결과를 `Data_set/Cluster_Results/`에 CSV로 저장
3. 전체 결과 JSON 저장

## 1. 설정 및 데이터 로딩

In [11]:
import pandas as pd
import numpy as np
import json
import os
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

from src import MultiHorizonPredictor, IndustryClusterer

print("모듈 로딩 완료")

모듈 로딩 완료


In [12]:
# 전체 데이터셋 로드 (Industry 컬럼이 포함된 주식 단위 데이터)
df_raw = pd.read_csv('Data_set/stock_features_clean.csv', parse_dates=['Date'])

print(f"데이터셋 로드: {len(df_raw):,} 행")
print(f"날짜 범위: {df_raw['Date'].min().date()} ~ {df_raw['Date'].max().date()}")
print(f"컬럼: {df_raw.columns.tolist()}")
print(f"\n섹터: {sorted(df_raw['Sector'].unique())}")
print(f"산업 수: {df_raw['Industry'].nunique()}")
print(f"기업 수: {df_raw['Company'].nunique()}")

데이터셋 로드: 603,359 행
날짜 범위: 2020-11-27 ~ 2026-01-09
컬럼: ['Date', 'Company', 'Sector', 'Industry', 'Open', 'High', 'Low', 'Close', 'Volume', 'Daily_Return_raw', 'Daily_Return_calc', 'Cum_Return', 'Return_1M', 'Return_3M', 'Return_6M', 'MA_5', 'MA_20', 'MA_60', 'Volatility_20d', 'Drawdown', 'MDD', 'DD_Short', 'Vol_MA_20', 'Vol_Ratio', 'Vol_Std_20', 'Vol_Z_Score', 'Log_Volume', 'Log_Volume_W', 'RSI_14', 'BB_Upper', 'BB_Middle', 'BB_Lower', 'BB_Width', 'Dividends', 'Stock Splits', 'Daily_Return', 'Cum_Max', 'Prev_Close', 'Gap', 'Gap_Pct', 'Is_Extreme_Change']

섹터: ['Basic Materials', 'Communication Services', 'Consumer Cyclical', 'Consumer Defensive', 'Energy', 'Financial Services', 'Healthcare', 'Industrials', 'Real Estate', 'Technology', 'Utilities']
산업 수: 100
기업 수: 481


## 2. 파라미터 설정

In [ ]:
# 예측 파라미터
PREDICTION_DATE = pd.Timestamp('2026-01-16')  # 원하는 예측 기준일로 변경
TRAIN_YEARS = 4  # 학습 데이터 연도 수

# 모델 파라미터 (백테스트 최적화 결과)
ALPHA = 0.6  # 하이브리드 모델 가중치
GAMMA = 0.5  # 랭킹에서 신뢰도 가중치
TOP_K = 3    # 호라이즌별 상위 섹터 수

# 인더스트리 클러스터링 파라미터
N_CLUSTERS = 5  # 클러스터 개수 (5개로 증가)

# 호라이즌별 lookback 기간 (일)
# 최소 60일 이상 필요 (통계적 신뢰성을 위한 충분한 거래일 확보)
HORIZON_LOOKBACK_MAP = {
    '1d': 60,     # 60일 (단기 트렌드)
    '3d': 75,     # 75일 (단기 트렌드)
    '1w': 90,     # 90일 (중단기 트렌드)
    '1m': 105,    # 105일 (중기 트렌드)
    '1q': 120,    # 120일 (중장기 트렌드)
    '6m': 150,    # 150일 (장기 트렌드)
    '1y': 180     # 180일 (장기 트렌드)
}

# 출력 디렉토리
OUTPUT_DIR = 'Data_set/Cluster_Results'
os.makedirs(OUTPUT_DIR, exist_ok=True)

print("파라미터:")
print(f"  예측 기준일: {PREDICTION_DATE.date()}")
print(f"  학습 연도: {TRAIN_YEARS}")
print(f"  Alpha: {ALPHA}")
print(f"  Gamma: {GAMMA}")
print(f"  Top-K: {TOP_K}")
print(f"  클러스터 수: {N_CLUSTERS} (5개 - 세밀한 분류)")
print(f"\n호라이즌별 특성 계산 기간 (차등화):")
for horizon, days in HORIZON_LOOKBACK_MAP.items():
    print(f"  {horizon}: {days}일 (~{int(days*0.7)} 거래일)")
print(f"\n출력 디렉토리: {OUTPUT_DIR}")

## 3. 멀티 호라이즌 예측을 위한 데이터 준비

In [14]:
# 섹터 단위로 집계 (일별 평균)
sector_df = df_raw.groupby(['Date', 'Sector'], as_index=False).agg({
    'Close': 'mean'
}).sort_values(['Sector', 'Date'])

print(f"섹터 집계: {len(sector_df):,} 행")

# 섹터 목록 추출
sectors = sorted(sector_df['Sector'].unique())
print(f"섹터: {sectors}")

섹터 집계: 14,135 행
섹터: ['Basic Materials', 'Communication Services', 'Consumer Cyclical', 'Consumer Defensive', 'Energy', 'Financial Services', 'Healthcare', 'Industrials', 'Real Estate', 'Technology', 'Utilities']


## 4. 멀티 호라이즌 예측

In [15]:
# 예측기 초기화
predictor = MultiHorizonPredictor(
    alpha=ALPHA,
    gamma=GAMMA,
    top_k=TOP_K
)

# 멀티 호라이즌 예측 실행
multi_horizon_results = predictor.predict_all_horizons(
    df=sector_df,
    prediction_date=PREDICTION_DATE,
    sectors=sectors,
    train_years=TRAIN_YEARS
)

15:47:05 - cmdstanpy - INFO - Chain [1] start processing



2026-01-16 기준 멀티 호라이즌 예측
학습 데이터: 2022-01-18 ~ 2026-01-09
호라이즌: ['1d', '3d', '1w', '1m', '1q', '6m', '1y']

[1d] 1일 호라이즌 예측 중...


15:47:05 - cmdstanpy - INFO - Chain [1] done processing
15:47:06 - cmdstanpy - INFO - Chain [1] start processing
15:47:06 - cmdstanpy - INFO - Chain [1] done processing
15:47:06 - cmdstanpy - INFO - Chain [1] start processing
15:47:06 - cmdstanpy - INFO - Chain [1] done processing
15:47:07 - cmdstanpy - INFO - Chain [1] start processing
15:47:07 - cmdstanpy - INFO - Chain [1] done processing
15:47:07 - cmdstanpy - INFO - Chain [1] start processing
15:47:07 - cmdstanpy - INFO - Chain [1] done processing
15:47:07 - cmdstanpy - INFO - Chain [1] start processing
15:47:08 - cmdstanpy - INFO - Chain [1] done processing
15:47:08 - cmdstanpy - INFO - Chain [1] start processing
15:47:08 - cmdstanpy - INFO - Chain [1] done processing
15:47:08 - cmdstanpy - INFO - Chain [1] start processing
15:47:09 - cmdstanpy - INFO - Chain [1] done processing
15:47:09 - cmdstanpy - INFO - Chain [1] start processing
15:47:09 - cmdstanpy - INFO - Chain [1] done processing
15:47:09 - cmdstanpy - INFO - Chain [1] 

  상위 3개 섹터: ['Healthcare', 'Real Estate', 'Basic Materials']

[3d] 3일 호라이즌 예측 중...


15:47:11 - cmdstanpy - INFO - Chain [1] done processing
15:47:11 - cmdstanpy - INFO - Chain [1] start processing
15:47:11 - cmdstanpy - INFO - Chain [1] done processing
15:47:11 - cmdstanpy - INFO - Chain [1] start processing
15:47:11 - cmdstanpy - INFO - Chain [1] done processing
15:47:12 - cmdstanpy - INFO - Chain [1] start processing
15:47:12 - cmdstanpy - INFO - Chain [1] done processing
15:47:12 - cmdstanpy - INFO - Chain [1] start processing
15:47:12 - cmdstanpy - INFO - Chain [1] done processing
15:47:13 - cmdstanpy - INFO - Chain [1] start processing
15:47:13 - cmdstanpy - INFO - Chain [1] done processing
15:47:13 - cmdstanpy - INFO - Chain [1] start processing
15:47:13 - cmdstanpy - INFO - Chain [1] done processing
15:47:13 - cmdstanpy - INFO - Chain [1] start processing
15:47:14 - cmdstanpy - INFO - Chain [1] done processing
15:47:14 - cmdstanpy - INFO - Chain [1] start processing
15:47:14 - cmdstanpy - INFO - Chain [1] done processing
15:47:14 - cmdstanpy - INFO - Chain [1] 

...

  상위 3개 섹터: ['Financial Services', 'Basic Materials', 'Industrials']

[6m] 180일 호라이즌 예측 중...


15:47:31 - cmdstanpy - INFO - Chain [1] done processing
15:47:31 - cmdstanpy - INFO - Chain [1] start processing
15:47:31 - cmdstanpy - INFO - Chain [1] done processing
15:47:32 - cmdstanpy - INFO - Chain [1] start processing
15:47:32 - cmdstanpy - INFO - Chain [1] done processing
15:47:32 - cmdstanpy - INFO - Chain [1] start processing
15:47:33 - cmdstanpy - INFO - Chain [1] done processing
15:47:33 - cmdstanpy - INFO - Chain [1] start processing
15:47:33 - cmdstanpy - INFO - Chain [1] done processing
15:47:33 - cmdstanpy - INFO - Chain [1] start processing
15:47:33 - cmdstanpy - INFO - Chain [1] done processing
15:47:34 - cmdstanpy - INFO - Chain [1] start processing
15:47:34 - cmdstanpy - INFO - Chain [1] done processing
15:47:34 - cmdstanpy - INFO - Chain [1] start processing
15:47:34 - cmdstanpy - INFO - Chain [1] done processing
15:47:35 - cmdstanpy - INFO - Chain [1] start processing
15:47:35 - cmdstanpy - INFO - Chain [1] done processing
15:47:35 - cmdstanpy - INFO - Chain [1] 

  상위 3개 섹터: ['Communication Services', 'Industrials', 'Financial Services']

[1y] 365일 호라이즌 예측 중...


15:47:36 - cmdstanpy - INFO - Chain [1] done processing
15:47:36 - cmdstanpy - INFO - Chain [1] start processing
15:47:37 - cmdstanpy - INFO - Chain [1] done processing
15:47:37 - cmdstanpy - INFO - Chain [1] start processing
15:47:37 - cmdstanpy - INFO - Chain [1] done processing
15:47:37 - cmdstanpy - INFO - Chain [1] start processing
15:47:38 - cmdstanpy - INFO - Chain [1] done processing
15:47:38 - cmdstanpy - INFO - Chain [1] start processing
15:47:38 - cmdstanpy - INFO - Chain [1] done processing
15:47:38 - cmdstanpy - INFO - Chain [1] start processing
15:47:39 - cmdstanpy - INFO - Chain [1] done processing
15:47:39 - cmdstanpy - INFO - Chain [1] start processing
15:47:39 - cmdstanpy - INFO - Chain [1] done processing
15:47:39 - cmdstanpy - INFO - Chain [1] start processing
15:47:40 - cmdstanpy - INFO - Chain [1] done processing
15:47:40 - cmdstanpy - INFO - Chain [1] start processing
15:47:40 - cmdstanpy - INFO - Chain [1] done processing
15:47:40 - cmdstanpy - INFO - Chain [1] 

  상위 3개 섹터: ['Industrials', 'Financial Services', 'Technology']

멀티 호라이즌 예측 완료


In [16]:
# 호라이즌별 상위 섹터 출력
print("\n" + "="*80)
print("호라이즌별 상위 섹터")
print("="*80)
for horizon, result in multi_horizon_results.items():
    print(f"\n{horizon} ({result['horizon_days']}일):")
    for i, sector in enumerate(result['top_sectors'], 1):
        pred = result['predictions'][sector]
        conf = result['confidences'][sector]
        score = result['ranking_scores'][sector]
        print(f"  {i}. {sector:25s} | 예측: {pred:7.2%} | 신뢰도: {conf:.3f} | 점수: {score:6.2f}")
print("="*80)


호라이즌별 상위 섹터

1d (1일):
  1. Healthcare                | 예측:   4.08% | 신뢰도: 0.467 | 점수:   1.91
  2. Real Estate               | 예측:   1.68% | 신뢰도: 0.510 | 점수:   0.92
  3. Basic Materials           | 예측:   2.51% | 신뢰도: 0.449 | 점수:   0.90

3d (3일):
  1. Financial Services        | 예측:   1.05% | 신뢰도: 0.476 | 점수:   1.69
  2. Basic Materials           | 예측:   0.62% | 신뢰도: 0.507 | 점수:   1.11
  3. Consumer Cyclical         | 예측:   0.68% | 신뢰도: 0.443 | 점수:   0.87

1w (7일):
  1. Financial Services        | 예측:   1.60% | 신뢰도: 0.478 | 점수:   1.19
  2. Basic Materials           | 예측:   1.26% | 신뢰도: 0.546 | 점수:   1.16
  3. Industrials               | 예측:   1.50% | 신뢰도: 0.438 | 점수:   0.85

1m (30일):
  1. Technology                | 예측:   5.83% | 신뢰도: 0.599 | 점수:   2.09
  2. Financial Services        | 예측:   5.25% | 신뢰도: 0.445 | 점수:   0.91
  3. Healthcare                | 예측:   5.37% | 신뢰도: 0.416 | 점수:   0.78

1q (90일):
  1. Financial Services        | 예측:   1.34% | 신뢰도: 0.570 | 점수:   1.14
  2. Basic M

## 5. 각 호라이즌별 인더스트리 클러스터링

In [17]:
# 클러스터러 초기화
clusterer = IndustryClusterer(
    n_clusters=N_CLUSTERS,
    random_state=42
)

# 각 호라이즌별로 클러스터링 수행
all_clustering_results = {}

date_str = PREDICTION_DATE.strftime('%Y%m%d')

for horizon_name, horizon_result in multi_horizon_results.items():
    print("\n" + "="*80)
    print(f"[{horizon_name}] 호라이즌 클러스터링 시작")
    print("="*80)
    
    # 해당 호라이즌의 Top-3 섹터
    top_sectors = horizon_result['top_sectors']
    print(f"선정된 섹터: {top_sectors}")
    
    # 호라이즌별 lookback 기간 가져오기
    lookback_days = HORIZON_LOOKBACK_MAP[horizon_name]
    print(f"특성 계산 기간: {lookback_days}일")
    
    # 클러스터링 수행
    clustering_result = clusterer.run_full_pipeline(
        df=df_raw,
        selected_sectors=top_sectors,
        lookback_days=lookback_days,  # 호라이즌별 lookback 사용
        end_date=PREDICTION_DATE
    )
    
    # 결과 저장
    all_clustering_results[horizon_name] = clustering_result
    
    # CSV 파일로 저장
    # 1. 전체 산업 특성
    df_industry = clustering_result['industry_features'].copy()
    df_industry.insert(0, 'Horizon', horizon_name)  # Horizon 컬럼 추가
    industry_filename = f"{OUTPUT_DIR}/{date_str}_{horizon_name}_industry_features.csv"
    df_industry.to_csv(industry_filename, index=False, encoding='utf-8-sig')
    print(f"\n저장: {industry_filename} ({len(df_industry)}개 산업)")
    
    # 2. 클러스터별 파일
    for cluster_name, df_cluster in clustering_result['by_cluster'].items():
        df_cluster_copy = df_cluster.copy()
        df_cluster_copy.insert(0, 'Horizon', horizon_name)  # Horizon 컬럼 추가
        cluster_filename = f"{OUTPUT_DIR}/{date_str}_{horizon_name}_{cluster_name}.csv"
        df_cluster_copy.to_csv(cluster_filename, index=False, encoding='utf-8-sig')
        print(f"저장: {cluster_filename} ({len(df_cluster_copy)}개 산업)")
    
    print(f"\n[{horizon_name}] 완료")

print("\n" + "="*80)
print("모든 호라이즌 클러스터링 완료")
print("="*80)


[1d] 호라이즌 클러스터링 시작
선정된 섹터: ['Healthcare', 'Real Estate', 'Basic Materials']
특성 계산 기간: 60일

산업 특성 추출 중...
기간: 2025-11-17 ~ 2026-01-16
섹터: ['Healthcare', 'Real Estate', 'Basic Materials']
산업 수: 25
25개 산업의 특성 추출 완료

KMeans 클러스터링 완료
클러스터 수: 4
클러스터 분포: {0: 15, 1: 6, 2: 1, 3: 3}

클러스터 프로파일 (수익률 기준 정렬)
         Return_3M  Volatility_20d     MDD  Sharpe_Ratio
cluster                                                 
3           0.2638          0.3421 -0.0569        4.8394
0           0.0694          0.1826 -0.0422        2.4647
1          -0.0283          0.1823 -0.0865       -1.4008
2          -0.1332          0.3219 -0.2222       -2.9429

클러스터 재매핑 (수익률 기준):
  Cluster 0: Return_3M = 26.38%
  Cluster 1: Return_3M = 6.94%
  Cluster 2: Return_3M = -2.83%
  Cluster 3: Return_3M = -13.32%

클러스터 해석 (수익률 기준 정렬)

CLUSTER_0: 공격형(고수익·고위험)
  수익률: 26.38%, 변동성: 34.21%, MDD: -5.69%
  샤프: 4.84
  산업 (3개): Gold, Chemicals, Copper...
  설명: 최고 수익률, 공격적 성장, 모멘텀 트레이딩에 적합

CLUSTER_1: 최적형(최적 리스크-보상)
  수익률: 6.94%, 변

## 6. 호라이즌별 클러스터 프로파일 요약

In [18]:
# 각 호라이즌별 클러스터 프로파일 출력
print("\n" + "="*80)
print("호라이즌별 클러스터 프로파일 요약")
print("="*80)

for horizon_name in multi_horizon_results.keys():
    clustering_result = all_clustering_results[horizon_name]
    
    print(f"\n{'='*80}")
    print(f"[{horizon_name}] - Top 섹터: {', '.join(multi_horizon_results[horizon_name]['top_sectors'])}")
    print(f"{'='*80}")
    
    cluster_profile = clustering_result['cluster_profile']
    print(cluster_profile)
    
    print(f"\n클러스터별 산업 수:")
    for cluster_name, df_cluster in clustering_result['by_cluster'].items():
        interp = clustering_result['interpretations'][cluster_name]
        print(f"  {cluster_name}: {len(df_cluster):2d}개 산업 - {interp['label']}")

print("\n" + "="*80)


호라이즌별 클러스터 프로파일 요약

[1d] - Top 섹터: Healthcare, Real Estate, Basic Materials
         Return_3M  Volatility_20d     MDD  Sharpe_Ratio
cluster                                                 
0           0.2638          0.3421 -0.0569        4.8394
1           0.0694          0.1826 -0.0422        2.4647
2          -0.0283          0.1823 -0.0865       -1.4008
3          -0.1332          0.3219 -0.2222       -2.9429

클러스터별 산업 수:
  cluster_0:  3개 산업 - 공격형(고수익·고위험)
  cluster_1: 15개 산업 - 최적형(최적 리스크-보상)
  cluster_2:  6개 산업 - 안정형(중립/저변동)
  cluster_3:  1개 산업 - 주의형(가치 함정)

[3d] - Top 섹터: Financial Services, Basic Materials, Consumer Cyclical
         Return_3M  Volatility_20d     MDD  Sharpe_Ratio
cluster                                                 
0           0.1956          0.1973 -0.0514        5.2705
1           0.1813          0.3857 -0.1090        2.6695
2           0.0685          0.1819 -0.0605        2.1032
3          -0.0497          0.2572 -0.1663       -0.9919

클러스터별 산업 수:
  c

## 7. 호라이즌별 상위 산업 출력

In [ ]:
# 각 호라이즌별로 강력 매수 클러스터(cluster_1)의 상위 산업 출력
print("\n" + "="*80)
print("호라이즌별 추천 산업 (강력 매수 - cluster_1)")
print("="*80)

for horizon_name in multi_horizon_results.keys():
    clustering_result = all_clustering_results[horizon_name]
    
    print(f"\n[{horizon_name}]")
    print("-" * 80)
    
    # cluster_1 (강력 매수) 찾기
    if 'cluster_1' in clustering_result['by_cluster']:
        df_best = clustering_result['by_cluster']['cluster_1']
        interp = clustering_result['interpretations']['cluster_1']
        
        print(f"클러스터: cluster_1 ({interp['label']})")
        print(f"샤프 비율: {interp['sharpe_ratio']:.2f}")
        print(f"\n상위 5개 산업:")
        
        for i, (idx, row) in enumerate(df_best.head(5).iterrows(), 1):
            print(f"  {i}. {row['Industry']:40s} | 섹터: {row['Sector']:20s} | 수익률: {row['Return_Period']:7.2%} | 샤프: {row['Sharpe_Ratio']:5.2f}")
    else:
        print("강력 매수 클러스터가 없습니다.")

print("\n" + "="*80)

## 9. 최종 요약

In [20]:
print("\n" + "="*80)
print("파이프라인 최종 요약")
print("="*80)
print(f"예측 기준일: {PREDICTION_DATE.date()}")
print(f"\n멀티 호라이즌 예측:")
print(f"  분석된 호라이즌: {len(multi_horizon_results)}개")
print(f"  호라이즌별 Top 섹터 수: {TOP_K}개")

print(f"\n호라이즌별 클러스터링 결과:")
for horizon_name in multi_horizon_results.keys():
    clustering_result = all_clustering_results[horizon_name]
    top_sectors = multi_horizon_results[horizon_name]['top_sectors']
    n_industries = len(clustering_result['industry_features'])
    
    print(f"\n  [{horizon_name}]:")
    print(f"    선정 섹터: {', '.join(top_sectors)}")
    print(f"    분석된 산업: {n_industries}개")
    print(f"    클러스터: {N_CLUSTERS}개")
    
    # 클러스터별 산업 수
    cluster_dist = {}
    for cluster_name, df_cluster in clustering_result['by_cluster'].items():
        cluster_dist[cluster_name] = len(df_cluster)
    print(f"    클러스터 분포: {cluster_dist}")

print(f"\n저장된 파일:")
print(f"  출력 디렉토리: {OUTPUT_DIR}")
print(f"  CSV 파일: {len(multi_horizon_results) * (1 + N_CLUSTERS)}개")
print(f"    - 호라이즌별 industry_features.csv: {len(multi_horizon_results)}개")
print(f"    - 호라이즌별 클러스터 CSV: {len(multi_horizon_results) * N_CLUSTERS}개")

print("\n" + "="*80)
print("완료")
print("="*80)


파이프라인 최종 요약
예측 기준일: 2026-01-16

멀티 호라이즌 예측:
  분석된 호라이즌: 7개
  호라이즌별 Top 섹터 수: 3개

호라이즌별 클러스터링 결과:

  [1d]:
    선정 섹터: Healthcare, Real Estate, Basic Materials
    분석된 산업: 25개
    클러스터: 4개
    클러스터 분포: {'cluster_0': 3, 'cluster_1': 15, 'cluster_2': 6, 'cluster_3': 1}

  [3d]:
    선정 섹터: Financial Services, Basic Materials, Consumer Cyclical
    분석된 산업: 32개
    클러스터: 4개
    클러스터 분포: {'cluster_0': 7, 'cluster_1': 4, 'cluster_2': 18, 'cluster_3': 3}

  [1w]:
    선정 섹터: Financial Services, Basic Materials, Industrials
    분석된 산업: 34개
    클러스터: 4개
    클러스터 분포: {'cluster_0': 4, 'cluster_1': 19, 'cluster_2': 9, 'cluster_3': 2}

  [1m]:
    선정 섹터: Technology, Financial Services, Healthcare
    분석된 산업: 31개
    클러스터: 4개
    클러스터 분포: {'cluster_0': 3, 'cluster_1': 17, 'cluster_2': 9, 'cluster_3': 2}

  [1q]:
    선정 섹터: Financial Services, Basic Materials, Industrials
    분석된 산업: 34개
    클러스터: 4개
    클러스터 분포: {'cluster_0': 8, 'cluster_1': 4, 'cluster_2': 16, 'cluster_3': 6}

  [6m]:
    선정 섹터: Commu